In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')

rat = pd.read_csv('ratings_small.csv', usecols=['userId', 'movieId', 'rating'])
rat.head()

mov = pd.read_csv('movies_metadata_fixed.csv', usecols = ['id', 'title'])

info = pd.merge(rat, mov, left_on='movieId', right_on='id')
info = info.drop('id' , 1)
info.head()

movie_ids = info['movieId'].unique()

def scale_movie_id(movie_id):
    scaled = np.where(movie_ids == movie_id)[0][0] + 1
    return scaled

info['movieId'] = info['movieId'].apply(scale_movie_id)

user_ids = info['userId'].unique()

def scale_user_id(user_id):
    scaled = np.where(user_ids == user_id)[0][0] + 1
    return scaled

info['userId'] = info['userId'].apply(scale_user_id)

n_movies = info['movieId'].nunique()
n_users = info['userId'].nunique()

info.head()




,userId,movieId,rating,title
0,1,1,2.5,Rocky III
1,2,1,4.0,Rocky III
2,3,1,3.0,Rocky III
3,4,1,4.0,Rocky III
4,5,1,3.0,Rocky III


In [2]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(info, test_size = 0.3)

from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, ground_truth):
    # Оставь оценки, предсказанные алгоритмом, только для нужного набора данных
    prediction = np.nan_to_num(prediction)[ground_truth.nonzero()].flatten()
    # Оставь действительные оценки пользователей только для соотвествующего набора данных
    ground_truth = np.nan_to_num(ground_truth)[ground_truth.nonzero()].flatten()
    mse = mean_squared_error(ground_truth, prediction)
    return sqrt(mse)



In [4]:
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]
    
test_data_matrix = np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

In [5]:
from  sklearn.metrics.pairwise import pairwise_distances

# считаем косинусное расстояние для пользователей и фильмов 
# (по строкам и по колонкам соотвественно).
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [6]:
#создаем датафрейм со случайными оценками
d = {"Хоббит":         pd.Series([0, 4, 0, 4], index=['Маша', 'Миша', 'Ваня', 'Настя']), 
     "Мстители":       pd.Series([5, 5, 0, 5], index=['Маша', 'Миша', 'Ваня', 'Настя']), 
     "Человек-паук":   pd.Series([1, 0, 1, 2], index=['Маша', 'Миша', 'Ваня', 'Настя']), 
     "Матрица":        pd.Series([5, 4, 2, 3], index=['Маша', 'Миша', 'Ваня', 'Настя']),
     "Звездные войны": pd.Series([0, 3, 1, 4], index=['Маша', 'Миша', 'Ваня', 'Настя'])}
df1 = pd.DataFrame(d)

display(df1)

,Хоббит,Мстители,Человек-паук,Матрица,Звездные войны
Маша,0,5,1,5,0
Миша,4,5,0,4,3
Ваня,0,0,1,2,1
Настя,4,5,2,3,4


In [7]:
dist = pairwise_distances(df1, metric='cosine')
dist = np.round(dist, 3)
dist_df = pd.DataFrame(data=dist, index = ['Маша', 'Миша', 'Ваня', 'Настя'], columns=['Маша', 'Миша', 'Ваня', 'Настя'])
display(dist_df)

,Маша,Миша,Ваня,Настя
Маша,0.000,0.224,0.371,0.297
Миша,0.224,0.000,0.447,0.044
Ваня,0.371,0.447,0.000,0.414
Настя,0.297,0.044,0.414,0.000


In [8]:
# User-based коллаборативная фильтрация
def naive_predict(top):
    # Структура хранения оценки фильмов от самых похожих пользователей для каждого зрителя
    # (количество похожих пользователей хранится в переменной top):
    # top_similar_ratings[0][1] - оценки всех фильмов одного из самых похожих пользователей на зрителя с id 0.
    # Здесь 1 - это не id пользователя, а просто порядковый номер.
    top_similar_ratings = np.zeros((n_users, top, n_movies))

    for i in range(n_users):
        # Для каждого зрителя необходимо найти наиболее похожих пользователей:
        # нулевой элемент не учитываем, так как на этом месте хранится похожесть пользователя самого на себя
        top_sim_users = user_similarity[i].argsort()[1:top + 1] #здесь в top_sim_users мы записываем id пользователей, у которых косинусные расстояния самые маленькие до текущего пользователя i
        
        # берём только оценки из "обучающей" выборки 
        top_similar_ratings[i] = train_data_matrix[top_sim_users] #создаём таблицу для каждого пользователя, в которой хранятся оценки за все фильмы от похожих пользователей

    pred = np.zeros((n_users, n_movies))
    for i in range(n_users):
        pred[i] =  top_similar_ratings[i].sum(axis=0) / top 
     
    return pred


def naive_predict_item(top):
    top_similar_ratings = np.zeros((n_movies, top, n_users))

    for i in range(n_movies):
        top_sim_movies = item_similarity[i].argsort()[1:top + 1] #находим ннаиболее близкие фильмы по оценкам

        top_similar_ratings[i] = train_data_matrix.T[top_sim_movies] #создаём таблицу для каждого фильма, в которой хранятся оценки всех пользователей за похожие фильмы 
        
    pred = np.zeros((n_movies, n_users))
    for i in range(n_movies):
        pred[i] = top_similar_ratings[i].sum(axis=0) / top
    
    return pred.T

naive_pred = naive_predict(7) #вызываем первую функцию, user-based, количество "похожих" пользователей - 7, передаём как аргумент
print('User-based CF RMSE: ', rmse(naive_pred, test_data_matrix))

naive_pred_item = naive_predict_item(7) #вызываем вторую функцию, item-based, количество "похожих" фильмов - 7, передаём как аргумент
print('Item-based CF RMSE: ', rmse(naive_pred_item, test_data_matrix))

User-based CF RMSE:  2.967391619552706
Item-based CF RMSE:  3.082788436412311


In [9]:
def k_fract_predict(top):
    top_similar = np.zeros((n_users, top))
    
    for i in range(n_users): #создаём массив, в котором будем хранить для каждого пользователя id "похожих" на него пользователей
        user_sim = user_similarity[i]
        top_sim_users = user_sim.argsort()[1:top + 1]#[-top:]

        top_similar[i] = top_sim_users
            
    pred = np.zeros((n_users, n_movies))
    
    for i in range(n_users):
        indexes = top_similar[i].astype(np.int) #записываем id людей, "похожих" на пользователя i в отдельную переменную
        numerator = user_similarity[i][indexes] #записываем в отдельную переменную вектор с косинусными расстояниями до ближайших "похожих" людей для пользователя i
        
        product = np.dot(numerator, train_data_matrix[indexes]) #Здесь мы реализуем вычисления для числителя в нашей формуле, но сразу для всех, перемножая между собой матрицы
        #первая матрица содержит в себе косинусное расстояние до ближайших "похожих" пользователей
        #вторая матрица содержит оценки этих пользователей за все фильмы
        
        denominator = numerator.sum() #сумма расстояний до "похожих" пользователей, наш знаменатель
        
        pred[i] = product / denominator
    
    return pred


def k_fract_predict_item(top):
    top_similar = np.zeros((n_movies, top))
    
    for i in range(n_movies): #создаём массив, в котором будем хранить для каждого фильма id "похожих" на него фильмов
        movies_sim = item_similarity[i]
        top_sim_movies = movies_sim.argsort()[1:top + 1]

        top_similar[i] = top_sim_movies.T
            
    pred = np.zeros((n_movies, n_users))
    
    
    for i in range(n_users):
        indexes = top_similar[i].astype(np.int) #записываем id фильмов, "похожих" на фильм i в отдельную переменную
        numerator = item_similarity[i][indexes] #записываем в отдельную переменную вектор с косинусными расстояниями до ближайших "похожих" фильмов для фильма i 
        
        product = np.dot(numerator, train_data_matrix.T[indexes])#Здесь мы реализуем вычисления для числителя в нашей формуле, но сразу для всех, перемножая между собой матрицы
        #первая матрица содержит в себе косинусное расстояние до ближайших "похожих" фильмов
        #вторая матрица содержит оценки всех пользователей за "похожие" фильмы
        
        denominator = numerator.sum()
        
        pred[i] = product / denominator
        
    return pred.T


k_predict = k_fract_predict(7) #вызываем первую функцию, user-based, количество "похожих" пользователей - 7, передаём как аргумент
print('User-based CF RMSE: ', rmse(k_predict, test_data_matrix))

k_predict_item = k_fract_predict_item(7) #вызываем вторую функцию, item-based, количество "похожих" фильмов - 7, передаём как аргумент
print('Item-based CF RMSE: ', rmse(k_predict_item, test_data_matrix))

User-based CF RMSE:  2.967618489698986
Item-based CF RMSE:  3.137197493368827


In [13]:
def k_fract_mean_predict(top):
    top_similar = np.zeros((n_users, top))
    
    for i in range(n_users):
        user_sim = user_similarity[i]
        top_sim_users = user_sim.argsort()[1:top + 1]
        
        top_similar[i] = top_sim_users

    pred = np.zeros((n_users, n_movies))
    
    for i in range(n_users):
        indexes = top_similar[i].astype(np.int)
        numerator = user_similarity[i][indexes]
        
        mean_rating = np.array([x for x in train_data_matrix[i] if x > 0]).mean() #вычисляем средний рейтинг пользователя i по всем фильмам, которые он посмотрел, если рейтинг равен 0, то не учитываем его
        
        diff_ratings = train_data_matrix[indexes] - train_data_matrix[indexes].mean() #находим разность между оценкой за конкретный фильм и средней оценкой по всем фильмам для всех "похожих" пользователей
        
        product = np.dot(numerator, diff_ratings)
        
        denominator =  numerator.sum()
      
        pred[i] =  mean_rating + product / denominator
        
    return pred
        
    return pred

def k_fract_mean_predict_item(top):
    top_similar = np.zeros((n_movies, top))
    
    for i in range(n_movies):
        movie_sim = item_similarity[i]
        top_sim_movies = movie_sim.argsort()[1:top + 1]
        
        top_similar[i] = top_sim_movies
           
    pred = np.zeros((n_movies, n_users))
    
    for i in range(n_movies):
        indexes = top_similar[i].astype(np.int)
        numerator =item_similarity[i][indexes]

        mean_rating = np.array([x for x in train_data_matrix.T[i] if x > 0]).mean()
        
        diff_ratings =train_data_matrix.T[indexes] - train_data_matrix.T[indexes].mean()
        
        product =  np.dot(numerator, diff_ratings)
        
        denominator = numerator.sum()
        
        pred[i] = mean_rating + product / denominator
        
    return pred.T

k_predict = k_fract_mean_predict(7) #вызываем первую функцию, user-based, количество "похожих" пользователей - 7, передаём как аргумент
print('User-based CF RMSE: ', rmse(k_predict, test_data_matrix))

k_predict_item = k_fract_mean_predict_item(7) #вызываем вторую функцию, item-based, количество "похожих" фильмов - 7, передаём как аргумент
print('Item-based CF RMSE: ', rmse(k_predict_item, test_data_matrix))

User-based CF RMSE:  1.3787389788241506
Item-based CF RMSE:  1.3940530548084307
